### Third try for widgets

See: https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html

and: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html

This notebook is a sandbox for exploring various techniques for developing widgets for design programs.

In [5]:
import inspect

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

from Designer import SST,show

from math import sqrt, pi

In [17]:
class What(object):   # haven't yet thought of a good name
    
    def __init__(self):
        self.section = None
        self.data = {}
        self.widgets = {}
        self.properties = set()
        self.widget_list = []
        self.output_widget = None
        self.main_widget = None
        self.dsg_widget = None
        self.compute_button = None
        self.defaults = {}
        
    def setDefaults(self,**kwargs):
        self.defaults.update(kwargs)
    
    def _build(self):
        self.output_widget = widgets.Output(layout={'border': '1px solid black'})
        with self.output_widget:
            b = widgets.Button(description='Compute!',tooltip='Click to perform calculations')
            b.on_click(self._on_compute_clicked)
            self.compute_button = b
            allwidg = widgets.VBox(self.widget_list+[b])
            self.main_widget = allwidg
    
    def _on_compute_clicked(self,b):
        """Examine the arguments of the callable and drag them out of data."""
        self.output_widget.clear_output()
        with self.output_widget:
            for key,w in self.widgets.items():
                self.data[key] = w.value
            self.callable(*[self.data[k] for k in self.callable_args])
            
    def _install_callable(self,callable):
        args = inspect.getfullargspec(callable)
        assert args.varargs is None and args.varkw is None
        if args.defaults:
            for k,v in zip(args.args[len(args.args)-len(args.defaults):],args.defaults):
                self.defaults[k] = v
        self.callable_args = args.args
        self.callable = callable
        
    def interact(self,callable):
        self._build()
        self._install_callable(callable)
        self._updateWidgetValues()
        display(self.main_widget)
        display(self.output_widget)
        
    def _updateWidgetValues(self):
        for k,w in self.widgets.items():
            if k in self.data:
                w.value = self.data[k]
            elif k in self.defaults:
                w.value = self.defaults[k]
        
    def _floatWidgets(self,text):
        ans = []
        for key in text.split(','):
            key = key.strip()
            val = 0.
            if '=' in key:
                k,v = key.split('=',1)
                key = k.strip()
                val = float(eval(v))
                self.defaults[key] = val
            w = widgets.FloatText(value=val,description=key+':')
            ans.append((key,w))
        return ans
    
    def _on_load_clicked(self,b):
        with self.output_widget:
            Dsg = self.dsg_widget.value
            section = SST.section(Dsg)
            for p in self.properties:
                self.data[p] = section[p]
                self._updateWidgetValues()
            self.data['Dsg'] = Dsg
            
    def defWidgets(self,text,properties=False):
        wlist = []
        if properties:
            dwidg = widgets.Text(value='W250x80',
                               placeholder='Shape Designation',
                               description='Dsg:',
                              tooltip='A shape designation to lookup for values.')
            pbut = widgets.Button(description='Load Properties',
                                   tooltip='Click to load properties into entries below.')
            pbut.on_click(self._on_load_clicked)
            top = widgets.HBox([dwidg,pbut])
            wlist.append(top)
            self.dsg_widget = dwidg
        for key,w in self._floatWidgets(text):
            self.widgets[key] = w
            if properties:
                self.properties.add(key)
            wlist.append(w)
        self.widget_list.extend(wlist)
        
    def show(self,text):
        show(text,data=self.data)

In [18]:
def do13_6(Dsg,Fy,Fu,Zx,Sx,Ix,Iy,J,Cw,D,B,T,W,L,w2=1):
    show('Dsg,Fy,Fu,*10^3,Zx,Sx,*10^6,Ix,Iy,*10^3,J,*10^9,Cw,*,D,B,T,W,L,w2')
    B_2T = (B/2.)/T
    fclass = 4
    if B_2T <= 145/sqrt(Fy):
        fclass = 1
    elif B_2T <= 170/sqrt(Fy):
        fclass = 2
    elif B_2T <= 200/sqrt(Fy):
        fclass = 3
    H_W = (D-2*T)/W
    wclass = 4
    if H_W <= 1100/sqrt(Fy):
        wclass = 1
    elif H_W <= 1700/sqrt(Fy):
        wclass = 2
    elif H_W <= 1900/sqrt(Fy):
        wclass = 3
    sclass = max(fclass,wclass)
    show('B/2T=B_2T,fclass,H/W=H_W,wclass,sclass')
    
    E = 200000.
    G = 77000.
    phi = 0.9
    
    if sclass > 3:
        raise Exception('Class 4 sections not yet implemented.')

    Mp = Zx * Fy * 1E-6
    My = Sx * Fy * 1E-6
    if L <= 0.:
        if sclass <= 2:
            Mr = phi*Mp
            show('Mp,Mr')
            return
        elif sclass == 3:
            Mr = phi*My
            show('My,Mr')
            return
    
    P = E*Iy*G*J
    Q = Iy*Cw*(pi*E/L)**2
    Mu = (w2*pi/L)*sqrt(P+Q) * 1E-6
    show('*10^21,P,Q,*,Mu')
    if sclass <= 2:
        if Mu >= 0.67*Mp:
            Mr = min(phi*Mp,1.15*phi*Mp*(1-0.28*Mp/Mu))
        else:
            Mr = phi*Mu
        show('Mp,Mr')
    elif sclass == 3:
        if Mu >= 0.67*My:
            Mr = min(phi*My,1.15*phi*My*(1-0.28*My/Mu))
        else:
            Mr = phi*Mu
        show('My,Mr')
    else:
        raise Exception('''See. I told you it wasn't implemented''')
    

In [19]:
w = What()
w.defWidgets('Fy=345.,Fu=450.')
w.defWidgets('Zx,Sx,Ix,Iy,J,Cw,D,B,T,W',properties=True)
w.defWidgets('L,w2')

w.interact(do13_6)

Output(layout=Layout(border='1px solid black'))

In [ ]:
w.output_widget.